<a href="https://colab.research.google.com/github/matheus896/crewai-examples/blob/main/Mentoria_CrewAI_Prof_Sandeco_Roteiro_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --q dotenv
!pip install --q litellm
!pip install --q crewai
!pip install --q 'crewai[tools]'

In [38]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from dotenv import load_dotenv
from crewai_tools import SerperDevTool, DallETool
from litellm import completion

In [90]:
# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Inicializar o modelo OpenAI
llm = LLM (
    model = 'gemini/gemini-1.5-flash',
    temperature=0.2
    )

#TOOLS

In [91]:
# Ferramenta de pesquisa na internet
search_tool = SerperDevTool()

#AGENTES

 - Pesquisador da web
 - Escritor de título
 - Roterista
 - Especialista em SEO
 - Revisor


#PESQUISADOR

In [92]:
pesquisador = Agent(
    role='Pesquisador de Tema',
    goal='Pesquisar informações detalhadas sobre {tema} na internet',
    backstory=(
        "Você é um especialista em pesquisa, com habilidades aguçadas para "
        "encontrar informações valiosas e detalhadas na internet."
    ),
    verbose=True,
    memory=True,
    tools=[search_tool],
    llm=llm
)

In [93]:
tarefa_pesquisa = Task(
    description=(
        "Pesquisar informações detalhadas e relevantes sobre o tema: {tema}. "
        "Concentre-se em aspectos únicos e dados importantes que podem enriquecer o vídeo. "
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Um documento com as principais informações e dados sobre {tema}.',
    tools=[search_tool],
    agent=pesquisador
)

#ESCRITOR TÍTULO

In [94]:
# 2. Agente Escritor de Títulos
escritor_titulos = Agent(
    role='Escritor de Títulos de Vídeo',
    goal='Criar títulos atraentes e otimizados para vídeos sobre {tema}',
    verbose=True,
    memory=True,
    backstory=(
        "Você tem uma habilidade especial para criar títulos que capturam a "
        "essência de um vídeo e atraem a atenção do público."
    ),
    llm=llm
)

In [95]:
tarefa_titulos = Task(
    description=(
        "Criar títulos atraentes e otimizados para o vídeo sobre o tema: {tema}. "
        "Certifique-se de que o título seja cativante e esteja otimizado para SEO. "
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Um título otimizado para o vídeo sobre {tema}.',
    agent=escritor_titulos,
    context = [tarefa_pesquisa]
)

#Roteirista

In [96]:
# 3. Agente Escritor de Roteiro
escritor_roteiro = Agent(
    role='Escritor de Roteiro',
    goal='Escrever um roteiro detalhado e envolvente para um vídeo sobre {tema}',
    verbose=True,
    memory=True,
    backstory=
        f"""Você é um contador de histórias talentoso, capaz de transformar
        informações em narrativas cativantes para vídeos."""
    ,
    llm=llm
)


In [97]:
# 3. Tarefa de Escrita de Roteiro
tarefa_roteiro = Task(
    description=(
        "Escrever um roteiro detalhado para o vídeo sobre o tema: {tema}. "
        "O roteiro deve ser envolvente e fornecer um fluxo lógico de informações. "
        "Se necessário, especifique imagens que podem enriquecer o conteúdo. "
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Um roteiro completo e bem estruturado para o vídeo sobre {tema}.',
    agent=escritor_roteiro,
    context = [tarefa_titulos, tarefa_pesquisa]

)

#Especialista em SEO

In [98]:
# 4. Agente Especialista em SEO para YouTube
especialista_seo = Agent(
    role='Especialista em SEO para YouTube',
    goal='Otimizar o roteiro e o título para que o vídeo tenha uma alta performance no YouTube',
    verbose=True,
    memory=True,
    backstory=(
        "Você é um expert em SEO, com um profundo entendimento das melhores "
        "práticas para otimizar conteúdo para o YouTube."
    ),
    llm=llm
)

In [99]:
# 4. Tarefa de SEO para YouTube
tarefa_seo = Task(
    description=(
        "Otimizar o título e o roteiro do vídeo para que tenha uma alta performance no YouTube. "
        "Incorporar as melhores práticas de SEO para garantir uma boa classificação e visibilidade. "
        "Crie hashtags, palavras-chaves e tags de vídeo para youtube. "
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Roteiro e título otimizados para o vídeo sobre {tema}, prontos para publicação no YouTube.',
    agent=especialista_seo,
    context = [tarefa_roteiro, tarefa_titulos, tarefa_pesquisa]
)

#Revisor

In [100]:
# 7. Agente Revisor
revisor = Agent(
    role='Revisor de Conteúdo',
    goal='Revisar todo o conteúdo produzido e entregar a versão final ao usuário',
    verbose=True,
    memory=True,
    backstory=(
        "Você tem um olho afiado para detalhes, garantindo que todo o conteúdo "
        "esteja perfeito antes de ser entregue ao usuário."
    ),
    llm=llm
)

In [101]:
# 7. Tarefa de Revisão e Entrega
tarefa_revisao = Task(
    description=(
        "Revisar todo o conteúdo produzido (título, roteiro, e otimização de SEO), "
        "preparar a versão final para entrega ao usuário."
        "Todo o texto deve estar em Português Brasil."
    ),
    expected_output='Conteúdo revisado, pronto para entrega ao usuário.',
    agent=revisor,
    output_file='output_teste.md', # Configurando o output para salvar em um arquivo Markdown
    context = [tarefa_seo, tarefa_roteiro, tarefa_titulos, tarefa_pesquisa]
)

#Crew

In [102]:
# Formando a crew
crew = Crew(
    agents=[
        pesquisador,
        escritor_titulos,
        escritor_roteiro,
        especialista_seo,
        revisor
    ],
    tasks=
    [
        tarefa_pesquisa,
        tarefa_titulos,
        tarefa_roteiro,
        tarefa_seo,
        tarefa_revisao
    ],
    process=Process.sequential, # Processamento sequencial das tarefas
    Verbose = True
)


#SOLICITAÇÃO DO TEMA AO USUÁRIO

In [103]:
tema_user = input("Digite o tema do vídeo: ")

Digite o tema do vídeo: Multi-agentes com CrewAI


In [104]:
print(tema_user)

Multi-agentes com CrewAI


In [ ]:
# Executando o processo com o tema escolhido
result = crew.kickoff(inputs={'tema': tema_user})

In [106]:
from IPython.display import Markdown
Markdown(result.raw)

## CrewAI: Multi-agentes para Automatizar Seus Processos e Impulsionar o Negócio!

**Introdução (0:00-0:30):**

* **Imagens:** Sequência rápida de imagens mostrando tarefas complexas e repetitivas em diversos setores (atendimento ao cliente, análise de dados, gerenciamento de estoque).
* **Narração:** "Em um mundo cada vez mais digital e acelerado, empresas buscam soluções para automatizar tarefas complexas e otimizar seus processos. Mas como lidar com a complexidade de fluxos de trabalho que exigem múltiplos passos e interações?"
* **Transição:** "A resposta está na inteligência artificial e na tecnologia de multi-agentes. Imagine um sistema onde agentes inteligentes, cada um com suas habilidades específicas, trabalham em conjunto para realizar tarefas complexas de forma eficiente e automatizada."

**Apresentação da CrewAI (0:30-1:00):**

* **Imagens:** Logo da CrewAI, animação gráfica mostrando a plataforma de código aberto.
* **Narração:** "CrewAI é uma plataforma inovadora que permite a criação e gerenciamento de sistemas de multi-agentes, utilizando inteligência artificial para revolucionar a automação de tarefas complexas. Com CrewAI, você pode criar agentes de IA personalizados, cada um com suas próprias habilidades e funções específicas."

**Funcionalidades da CrewAI (1:00-1:45):**

* **Imagens:** Animação gráfica mostrando os recursos da CrewAI:
    * **Criação de Agentes:** Mostrar a personalização de agentes com diferentes habilidades e funções.
    * **Orquestração de Agentes:** Demonstrar a coordenação de múltiplos agentes trabalhando em conjunto.
    * **Comunicação entre Agentes:** Visualizar a troca de informações entre os agentes.
    * **Gerenciamento de Memória:** Mostrar como os agentes armazenam e acessam informações.
    * **Integração com Ferramentas Externas:** Exibir a conexão com buscadores web, modelos de linguagem, etc.
    * **Funcionalidades Avançadas:** Destacar recursos como webhooks, suporte gRPC e integração com APIs.
* **Narração:** "Esses agentes podem se comunicar entre si, compartilhar informações e colaborar para alcançar objetivos comuns. A plataforma oferece recursos avançados de gerenciamento de memória, integração com ferramentas externas e funcionalidades que garantem a robustez e escalabilidade do sistema."

**Aplicações Práticas (1:45-2:30):**

* **Imagens:** Mostrar exemplos práticos de como a CrewAI pode ser utilizada em diferentes setores:
    * **Automação de Processos de Negócios:** Mostrar a otimização de tarefas como análise de dados, geração de relatórios e atendimento ao cliente.
    * **Desenvolvimento de Chatbots Inteligentes:** Demonstrar a criação de chatbots que conversam com usuários de forma natural e personalizada.
    * **Geração de Conteúdo Automatizado:** Exibir a criação automática de artigos, posts de blog e descrições de produtos.
    * **Pesquisa e Análise de Dados:** Mostrar a coleta, análise e interpretação automatizada de dados.
* **Narração:** "CrewAI oferece uma solução completa para automatizar tarefas complexas em diversos setores. Desde a otimização de processos de negócios até a criação de chatbots inteligentes e a geração de conteúdo automatizado, a plataforma oferece um leque de possibilidades para impulsionar a eficiência e a produtividade."

**Benefícios da CrewAI (2:30-3:00):**

* **Imagens:** Mostrar os benefícios do uso da CrewAI:
    * **Automação de Tarefas Complexas:** Destacar a liberação de tempo e recursos para outras atividades.
    * **Aumento da Eficiência:** Mostrar a otimização do desempenho das tarefas e a redução de erros.
    * **Escalabilidade:** Demonstrar a capacidade de adaptação do sistema às necessidades de diferentes projetos.
    * **Flexibilidade:** Mostrar a personalização de agentes para atender às necessidades específicas de cada aplicação.
* **Narração:** "Com CrewAI, você pode automatizar tarefas complexas, aumentar a eficiência, escalar seus sistemas e personalizar soluções para atender às necessidades específicas do seu negócio."

**Considerações Importantes (3:00-3:30):**

* **Imagens:** Mostrar os desafios e considerações importantes ao utilizar CrewAI:
    * **Complexidade:** Mostrar a necessidade de conhecimento técnico em áreas como inteligência artificial e programação.
    * **Custos:** Destacar os custos com infraestrutura, desenvolvimento e treinamento de agentes.
    * **Segurança:** Enfatizar a importância da segurança dos dados e a proteção contra acesso não autorizado.
* **Narração:** "É importante lembrar que a criação de sistemas de multi-agentes pode ser complexa e envolver custos. A segurança dos dados também é crucial e deve ser priorizada."

**Conclusão e Chamada para Ação (3:30-4:00):**

* **Imagens:** Mostrar o site da CrewAI, com um call to action para os espectadores.
* **Narração:** "CrewAI é uma plataforma promissora que abre portas para um futuro de automação inteligente, onde a inteligência artificial colabora para alcançar resultados inovadores. Explore as possibilidades e descubra como CrewAI pode revolucionar o seu negócio. Visite o site da CrewAI para saber mais sobre a plataforma e como ela pode ajudar você a automatizar seus processos e impulsionar o seu negócio."

**Textos na Tela:**

* **CrewAI: Multi-agentes para automatizar seus processos e impulsionar o negócio!**
* **Crie agentes de IA personalizados e transforme seus fluxos de trabalho.**
* **Domine a automação com CrewAI: Guia completo de multi-agentes.**
* **CrewAI: A plataforma de código aberto que revoluciona a automação de tarefas complexas.**
* **Multi-agentes com CrewAI: Potencialize a eficiência e a escalabilidade do seu negócio.**

**Chamadas para ação:**

* Visite o site da CrewAI para saber mais.
* Baixe o guia completo de multi-agentes com CrewAI.
* Experimente a plataforma gratuitamente.

**Hashtags:**

* #CrewAI
* #MultiAgentes
* #Automação
* #InteligênciaArtificial
* #IA
* #Tecnologia
* #Negócios
* #Eficiência
* #Produtividade
* #Inovação
* #Futuro
* #CódigoAberto
* #Chatbots
* #GeraçãoDeConteúdo
* #AnáliseDeDados

**Tags de Vídeo:**

* CrewAI
* Multi-agentes
* Automação de processos
* Inteligência artificial
* Plataforma de código aberto
* Chatbots inteligentes
* Geração de conteúdo automatizado
* Análise de dados
* Eficiência e produtividade
* Escalabilidade e flexibilidade
* Segurança de dados
* Benefícios da CrewAI
* Considerações importantes
* Call to action

**Observações:**

* O roteiro pode ser adaptado de acordo com a duração desejada do vídeo.
* As imagens podem ser substituídas por outras que sejam mais adequadas ao tema.
* É importante utilizar uma linguagem clara e objetiva, com foco em apresentar os benefícios e funcionalidades da CrewAI de forma fácil e compreensível.
* A música de fundo e os efeitos sonoros devem ser escolhidos de forma a criar uma atmosfera positiva e envolvente.

**Palavras-chave:**

* CrewAI
* Multi-agentes
* Automação
* Inteligência Artificial
* IA
* Plataforma de código aberto
* Chatbots
* Geração de conteúdo
* Análise de dados
* Eficiência
* Produtividade
* Escalabilidade
* Flexibilidade
* Segurança
* Benefícios
* Considerações
* Call to action

**Dicas:**

* Use imagens de alta qualidade e animações atraentes para manter o público engajado.
* Inclua um call to action claro e conciso no final do vídeo.
* Promova seu vídeo nas redes sociais e em outros canais relevantes.
* Monitore o desempenho do seu vídeo e faça ajustes conforme necessário.

Espero que este roteiro otimizado e as dicas adicionais ajudem você a criar um vídeo de alta performance sobre CrewAI no YouTube!